In [21]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

In [22]:
def classical_edge_detect(image, threshold_x, threshold_y, shift=True):
    h, w = image.shape
    edge_map = np.zeros_like(image)

    for y in range(h):
        for x in range(w):
            val = image[y, x]

            # X-gradient
            if x + 1 < w:
                gx = int(image[y, (x + 1) % w]) - int(val)
                if abs(gx) >= threshold_x:
                    dx = 1 if (shift and gx < 0) else 0
                    edge_map[y, (x + dx) % w] = 1

            # Y-gradient
            if y + 1 < h:
                gy = int(image[(y + 1) % h, x]) - int(val)
                if abs(gy) >= threshold_y:
                    dy = 1 if (shift and gy < 0) else 0
                    edge_map[(y + dy) % h, x] = 1

    return edge_map


In [23]:
def run_all_threshold_pairs(image, intensity_bits, output_dir="classical_edges", visualize=False):
    os.makedirs(output_dir, exist_ok=True)
    max_threshold = 2 ** (intensity_bits)

    for tx in range(max_threshold):
        for ty in range(max_threshold):
            edge_map = classical_edge_detect(image, threshold_x=tx, threshold_y=ty)

            # Create figure
            plt.figure(figsize=(4, 4))
            plt.imshow(edge_map, cmap='gray', interpolation='nearest')
            plt.colorbar(label='Edge')
            plt.title(f'Edge Map\nThresholds: X={tx}, Y={ty}')
            plt.axis('off')

            # Save with full figure context
            filename = f"{output_dir}/edge_tx{tx:02d}_ty{ty:02d}.png"
            plt.savefig(filename, bbox_inches='tight')
            plt.close()

            if visualize:
                plt.show()

    print(f"Saved {max_threshold**2} full-size visualizations to {output_dir}")


In [24]:
def load_image(filepath, size=(16, 16)):
    img = Image.open(filepath).convert('L')  # Grayscale
    img = img.resize(size, Image.Resampling.LANCZOS)
    img_array = np.array(img)
    return img_array

def normalize_image(image, bits=3):
    max_val = 2**bits - 1
    return np.round((image / 255) * max_val).astype(int)


image = load_image('images/batman16.jpg', size=(64, 64))
image = normalize_image(image,4)
print(image)

[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]]


In [25]:
run_all_threshold_pairs(image, intensity_bits=4, output_dir="classical_edges")

Saved 256 full-size visualizations to classical_edges
